### Imports & installation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
pip uninstall holidays -y

In [ ]:
pip install holidays==0.23

In [ ]:
pip install prophet

In [ ]:
from prophet import Prophet

### Loading & transforming the data

In [ ]:
def to_datetime(df):
    # Changes date to datetime 
    for index, value in enumerate(df['Month']):
        df.at[index, 'Month'] = datetime.strptime(value, '%Y-%m')
    return df

def get_most_frequent_locations(df):
    # Returns a list of most frequent location per month 
    df_grouped = df.groupby([pd.Grouper(key='Month', freq='M'), 'LSOA code']).size()
    most_frequent_location = df_grouped.groupby(level=0).idxmax()
    locs = []
    for loc in range(len(most_frequent_location)):
        locs.append(most_frequent_location[loc][1])
    return locs

def count_per_month(df):
    # Return a dataframe with crimes count per month
    df_per_month = df.groupby(pd.Grouper(key='Month', freq='M')).size()
    df_per_month = pd.DataFrame(df_per_month)
    df_per_month['ds'] = df_per_month.index
    df_per_month = df_per_month.rename(columns={0: 'y'})
    return df_per_month 

def add_locs(df, locs):
    # Adds most frequent location column to the dataframe
    df['loc'] = locs
    return df

In [ ]:
def loc_encoding(df):
    # Returns a dataframe with encoded locations
    one_hot_encoded = pd.get_dummies(df['loc'])
    df_encoded = pd.concat([df, one_hot_encoded], axis=1)
    df_encoded = df_encoded.drop('loc', axis=1)
    return df_encoded

def get_cols_for_pred(df):
    # Returns a list of additional columns for regression 
    columns = df.iloc[:, 2:]
    return columns

In [ ]:
train_data = pd.read_csv('burglary_train.csv')
test_data = pd.read_csv('burglary_test.csv')

In [ ]:
train_data = to_datetime(train_data)
locs = get_most_frequent_locations(train_data)
train_data = count_per_month(train_data)
train_data = add_locs(train_data, locs)
train_data = loc_encoding(train_data)
pred_cols = get_cols_for_pred(train_data)

In [ ]:
test_data = to_datetime(test_data)
locs = get_most_frequent_locations(test_data)
test_data = count_per_month(test_data)
test_data = add_locs(test_data, locs)
test_data = loc_encoding(test_data) 

# Prophet

### Basic implementation on aggregated data

In [ ]:
prophet_model = Prophet(yearly_seasonality=True)

prophet_model.fit(train_data)

future = prophet_model.make_future_dataframe(periods=12, freq='M')
predictions = prophet_model.predict(future)

In [ ]:
plt = prophet_model.plot(predictions)
plt.legend()
plt.show()

In [ ]:
test_predictions = predictions[-12:]
test_predictions_reset = test_predictions.reset_index(drop=True)
test_data_reset = test_data[-12:].reset_index(drop=True)

In [ ]:
mse = ((test_predictions_reset['yhat'] - test_data_reset['y']) ** 2).mean()
rmse = mse ** 0.5
print('RMSE: {:.2f}'.format(rmse))

In [ ]:
components = prophet_model.plot_components(predictions)

#### On aggregated data with most common location per month as and additional regressor

In [ ]:
# new model with location as additional regressor (in dummy variables)
prophet_model_extra = Prophet(yearly_seasonality=True)

# adds all location dummies to the model and fits the training data
for col in pred_cols:
    prophet_model_extra.add_regressor(col)

prophet_model_extra.fit(train_data)

In [ ]:
# extract column names. This is needed to create the future dataset
colnames = pred_cols.columns.values.tolist()

In [ ]:
# create future dataframe, merge it with the columns where the date matches
future_extra = prophet_model_extra.make_future_dataframe(periods=12, freq='M')
future_extra = train_data[['ds']+colnames].merge(future_extra, how='outer', on='ds')
future_extra

In [ ]:
forecast = prophet_model.predict(future_extra) 

In [ ]:
plt = prophet_model_extra.plot(forecast)
plt.legend()
plt.show()

In [ ]:
test_predictions = forecast[-12:]
test_predictions_reset = test_predictions.reset_index(drop=True)
test_data_reset = test_data[-12:].reset_index(drop=True)

In [ ]:
mse = ((test_predictions_reset['yhat'] - test_data_reset['y']) ** 2).mean()
rmse = mse ** 0.5
print('RMSE: {:.2f}'.format(rmse))

The plot differs a bit from the first one, but RMSE did not improve.

### Per location (ward and LSOA), with aggregation (monthly counts)

In [ ]:
def drop(df):
    # Dropping columns (which are not encoded and not used for further predictions)
    df = df.drop(df.columns[0], axis=1)
    df = df.drop(['Longitude', 'Latitude', 'Location', 'LSOA name', 'Last outcome category'], axis=1)
    return df

In [ ]:
def loc_enc(df):
    # Returns a dataframe with encoded locations (based on LSOA code)
    one_hot_encoded = pd.get_dummies(df['LSOA code'])
    df_encoded = pd.concat([df, one_hot_encoded], axis=1)
    df_encoded = df_encoded.drop('LSOA code', axis=1)
    return df_encoded

In [ ]:
def ward_enc(df):
    # Returns a dataframe with encoded locations (based on ward name)
    one_hot_encoded = pd.get_dummies(df['Ward name'])
    df_encoded = pd.concat([df, one_hot_encoded], axis=1)
    df_encoded = df_encoded.drop('Ward name', axis=1)
    return df_encoded

In [ ]:
train_data = pd.read_csv('burglary_train.csv')
test_data = pd.read_csv('burglary_test.csv')
all_wards = train_data['Ward name'].unique()
train_data = to_datetime(train_data)
test_data = to_datetime(test_data)
train_data = drop(train_data)
test_data = drop(test_data)
train_data = ward_enc(train_data)
test_data = ward_enc(test_data)

In [ ]:
more_train = test_data.iloc[:6325, :]
new_test = test_data.iloc[6325:, :]
merged = pd.concat([train_data, more_train], axis=0)
train_data = merged
test_data = new_test

In [ ]:
# new dataframe to not to get confused
df_prophet = pd.DataFrame()
test_prophet = pd.DataFrame()

df_prophet['ds'] = train_data['Month']
test_prophet['ds'] = test_data['Month']

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

### For each ward

In [ ]:
ward_df = pd.DataFrame(columns=['rmse', 'mae', 'r-squared'])
ward_df.loc[0] = [0,0,0]

In [ ]:
maes = 0
rmses = 0 
rsquareds = 0

# creating models and calculating performance metrics per ward

for ward in all_wards:
    
    train_data = pd.read_csv('burglary_train.csv')
    test_data = pd.read_csv('burglary_test.csv')

    train_data = to_datetime(train_data)
    test_data = to_datetime(test_data)
    train_data = drop(train_data)
    test_data = drop(test_data)
    train_data = ward_enc(train_data)
    test_data = ward_enc(test_data)
    
    more_train = test_data.iloc[:6325, :]
    new_test = test_data.iloc[6325:, :]
    merged = pd.concat([train_data, more_train], axis=0)
    train_data = merged
    test_data = new_test

    df_prophet = pd.DataFrame()
    test_prophet = pd.DataFrame()

    df_prophet['ds'] = train_data['Month']
    test_prophet['ds'] = test_data['Month']
    df_prophet['y'] = train_data[ward]
    test_prophet['y'] = test_data[ward]
    
    df_prophet.set_index('ds', inplace=True)
    df_prophet = df_prophet.resample('M').sum()
    df_prophet['ds'] = df_prophet.index

    test_prophet.set_index('ds', inplace=True)
    test_prophet = test_prophet.resample('M').sum()
    test_prophet['ds'] = test_prophet.index

    train_data = df_prophet
    test_data = test_prophet
    
    # changepoint_prior_scale can be added in the Prophet brackets to check the tuning results
    model = Prophet()
    model.fit(train_data)
    
    future_dates = model.make_future_dataframe(periods=12, freq='M')
    predictions = model.predict(future_dates)
    
    test_predictions = predictions[-12:]
    test_predictions_reset = test_predictions.reset_index(drop=True)
    test_data_reset = test_data[-12:].reset_index(drop=True)
    
    mse = ((test_predictions_reset['yhat'] - test_data_reset['y']) ** 2).mean()
    rmse = mse ** 0.5
    
    test_predictions_reset['yhat'] = test_predictions_reset['yhat'].astype(int)
    
    predicted_values = predictions['yhat'].tail(12)
    actual_values = test_data_reset['y']

    mae = mean_absolute_error(actual_values, predicted_values)
    r_squared = r2_score(actual_values, predicted_values)
    
    maes = maes + mae
    rmses = rmses + rmse
    rsquareds = rsquareds + r_squared
    ward_df.loc[ward] = [round(rmse, 2), mae.round(2), r_squared.round(2)]

In [ ]:
print("mae: ", maes/len(all_wards), ", rmse: ", rmses/len(all_wards), ", r-squared: ", rsquareds/len(all_wards))

In [ ]:
ward_df.drop(index=0).to_csv('results_ward.csv', index_label = 'ward')

### For each LSOA

In [ ]:
train_data = pd.read_csv('burglary_train.csv')
test_data = pd.read_csv('burglary_test.csv')
train_data = to_datetime(train_data)
test_data = to_datetime(test_data)
train_data = drop(train_data)
test_data = drop(test_data)
train_data = loc_enc(train_data)
test_data = loc_enc(test_data)
more_train = test_data.iloc[:6325, :]
new_test = test_data.iloc[6325:, :]
merged = pd.concat([train_data, more_train], axis=0)
train_data = merged
test_data = new_test

In [ ]:
# creating a dataframe to save the results
results_sum = pd.DataFrame({'LSOA':[], 'RMSE':[], 'MAE':[], 'R^2':[]})
i = 0

In [ ]:
maes = 0
rmses = 0
rsquareds = 0

# going though lsoas in encoded columns (first one is ds, thus is omitted)
rmse_mean = []
for lsoa in train_data.columns[50:70]:

    # new dataframe to not to get confused
    df_prophet = pd.DataFrame()
    test_prophet = pd.DataFrame()

    # adding ds and y columns for prophet
    df_prophet['ds'] = train_data['Month']
    df_prophet['y'] = train_data[lsoa]
    test_prophet['ds'] = test_data['Month']
    test_prophet['y'] = test_data[lsoa]

    df_prophet.set_index('ds', inplace=True)
    df_prophet = df_prophet.resample('M').sum()
    df_prophet['ds'] = df_prophet.index

    test_prophet.set_index('ds', inplace=True)
    test_prophet = test_prophet.resample('M').sum()
    test_prophet['ds'] = test_prophet.index

    # changepoint_prior_scale can be added in the Prophet brackets to check the tuning results
    model = Prophet()
    model.fit(df_prophet)

    future_dates = model.make_future_dataframe(periods=12, freq='M')
    predictions = model.predict(future_dates)
    plt = model.plot(predictions)
    plt.legend()
    plt.show()

    test_predictions = predictions[-12:]
    test_predictions_reset = test_predictions.reset_index(drop=True)
    test_data_reset = test_prophet[-12:].reset_index(drop=True)

    mse = ((test_predictions_reset['yhat'] - test_data_reset['y']) ** 2).mean()
    rmse = mse ** 0.5

    test_predictions_reset['yhat'] = test_predictions_reset['yhat'].astype(int)

    predicted_values = predictions['yhat'].tail(12)
    actual_values = test_data_reset['y']

    mae = mean_absolute_error(actual_values, predicted_values)
    r_squared = r2_score(actual_values, predicted_values)

    # adding to the dataframe
    results_sum.loc[i] = lsoa, round(rmse, 2), mae.round(2), r_squared.round(2)
    i += 1
    rmse_mean.append(rmse)
    
    maes = maes + mae
    rmses = rmses + rmse
    rsquareds = rsquareds + r_squared

print('Mean RMSE per LSOA: {}'.format(sum(rmse_mean)/len(rmse_mean)))

In [ ]:
print("mae: ", maes/20, ", rmse: ", rmses/20, ", r-squared: ", rsquareds/20)

In [ ]:
results_sum.to_csv('results.csv', index=False)

In [ ]:
min_rmse = results_sum['RMSE'].min()
max_rmse = results_sum['RMSE'].max()
print(min_rmse, max_rmse)

In [ ]:
good_lsoa = results_sum[results_sum['RMSE'] == min_rmse]
bad_lsoa = results_sum[results_sum['RMSE'] == max_rmse]
good_lsoa